# Demo - Download video from Youtube and Extract audio from video

## Install moviepy

In [3]:
!pip install moviepy

## Install yt-dlp from github

In [9]:
!pip install git+https://github.com/yt-dlp/yt-dlp.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Download video from Youtube and Extract audio from video


In [16]:
import yt_dlp
from moviepy.editor import VideoFileClip
import logging
from pathlib import Path
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_youtube_video(url: str, output_dir: str = "downloads", video_name="video") -> str:
    """
    Download YouTube video using yt-dlp with error handling and format selection
    Returns path to downloaded video file
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'merge_output_format': 'mp4',
        'outtmpl': f'{output_dir}/{video_name}.mp4',
        'restrictfilenames': True,
        'quiet': True,
        'no_warnings': True,
        'logger': logging.getLogger('yt-dlp'),
        'concurrent_fragment_downloads': 3,
        'retries': 5,
        'fragment_retries': 5,
        'extractor_retries': 3,
        'socket_timeout': 30,
        'http_headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
        }
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=False)
            final_path = Path(output_dir) / f"{video_name}.mp4"

            if final_path.exists():
                logging.warning(f"File already exists: {final_path}")
                return str(final_path)

            logging.info(f"Downloading: {info_dict['title']} [ID: {info_dict['id']}]")
            ydl.download([url])

            # Verify successful download
            if not final_path.exists():
                raise FileNotFoundError("Downloaded file not found - possible download failure")

            return str(final_path)

    except Exception as e:
        logging.error(f"Download failed: {str(e)}")
        raise

def extract_audio(video_path: str, audio_name:str, output_dir: str = "downloads", output_format: str = 'wav', bitrate: str = '320k') -> str:
    """
    Extract audio using moviepy with high quality settings
    Returns path to extracted audio file
    """
    audio_output = Path(f"{output_dir}/{audio_name}").with_suffix(f'.{output_format}')

    try:
        with VideoFileClip(video_path) as video:
            if not video.audio:
                raise ValueError("No audio track detected in video file")

            # Audio settings
            if output_format == 'wav':
                audio = video.audio
                audio.write_audiofile(
                    str(audio_output),
                    codec='pcm_s16le',
                    fps=44100,
                    ffmpeg_params=['-ac', '2', '-ar', '44100']
                )
            else:
                video.audio.write_audiofile(
                    str(audio_output),
                    codec='mp3' if output_format == 'mp3' else 'aac',
                    bitrate=bitrate
                )

        return str(audio_output)

    except Exception as e:
        logging.error(f"Audio extraction failed: {str(e)}")
        raise
    finally:
        if 'video' in locals():
            video.close()

def main():
    video_url = "https://www.youtube.com/watch?v=vsbmrmHuZQ4"

    try:
        # Download video
        video_file = download_youtube_video(video_url, video_name="Trump_video", output_dir="/content/drive/MyDrive/data/youtube_downloads")
        logging.info(f"Successfully downloaded: {video_file}")

        # Extract audio (WAV format by default)
        audio_file = extract_audio(video_file, audio_name="Trump_audio", output_dir="/content/drive/MyDrive/data/youtube_downloads", output_format='wav')
        logging.info(f"Successfully extracted audio: {audio_file}")

        # Optional: Remove original video
        # os.remove(video_file)

    except Exception as e:
        logging.critical(f"Processing failed: {str(e)}")
        exit(1)

if __name__ == "__main__":
    main()

MoviePy - Writing audio in /content/drive/MyDrive/data/youtube_downloads/Trump_audio.wav


MoviePy - Done.
